In [13]:
import io
import pandas as pd
import requests
import csv
import re

In [14]:
FMP_API_ENDPOINT="https://financialmodelingprep.com/api"
FMP_API_KEY="6b5ead8d3c6bceb25d50bc6237dc8543"

In [43]:
def process_fmp_financial_statements(file_path):
    raw_df = pd.read_csv(file_path)
    raw_df.rename({"Unnamed: 1": "metric"}, axis=1, inplace=True)
    df = raw_df[~raw_df['metric'].isna()]
    df = df.drop("date", axis=1)
    df = df.rename({"metric": "date"}, axis=1)
    df = df.set_index("date").transpose()
    df.index = df.index.map(lambda s: re.sub(r"_Q[1-4]", "", s))
    df = df.sort_index(ascending=False)
    return df


In [21]:
def pull_fmp_financial_statements(stock_symbols, output_dir):
    statment_types = ["income-statement", "balance-sheet-statement", "cash-flow-statement"]
    session = requests.Session()
    for sym in stock_symbols:
        for stmt_type in statment_types:
            statement_url = f"{FMP_API_ENDPOINT}/v3/{stmt_type}/{sym}"
            resp = session.get(url = statement_url, params={
                    "apikey": FMP_API_KEY,
                    "period": "quarter",
                    "limit": 1,
                    "datatype": "csv"
                })
            if resp.status_code != 200:
                raise RuntimeError(f"http status is {resp.status_code}")
            file_name = f"{output_dir}/{sym}-{stmt_type}"
            raw_file_path = f"{file_name}-raw.csv"
            processed_file_path = f"{file_name}.csv"
            with open(raw_file_path, 'w', encoding='utf-8') as f:
                writer = csv.writer(f)
                reader = csv.reader(resp.content.decode('utf-8').splitlines())
                for row in reader:
                    if len(row) >= 3 and len(row[2]) > 0:
                        writer.writerow(row)
            df = process_fmp_financial_statements(raw_file_path)
            df.to_csv(processed_file_path)


In [44]:
output_dir="../../output"
# pull_fmp_financial_statements(["AAPL"], output_dir)

process_fmp_financial_statements(file_path="../../output/AAPL-balance-sheet-statement-raw.csv")

date,cashAndCashEquivalents,shortTermInvestments,cashAndShortTermInvestments,netReceivables,inventory,otherCurrentAssets,totalCurrentAssets,propertyPlantEquipmentNet,goodwill,intangibleAssets,...,totalLiabilities,commonStock,retainedEarnings,accumulatedOtherComprehensiveIncomeLoss,othertotalStockholdersEquity,totalStockholdersEquity,totalLiabilitiesAndStockholdersEquity,totalInvestments,totalDebt,netDebt
2022-12-31,2.053500e+10,3.082000e+10,5.135500e+10,5.418000e+10,6.820000e+09,1.642200e+10,1.287770e+11,4.295100e+10,0.0,0.0,...,2.900200e+11,6.639900e+10,3.240000e+09,-1.291200e+10,0.0,5.672700e+10,3.467470e+11,1.449150e+11,1.111100e+11,9.057500e+10
2022-09-24,2.364600e+10,2.465800e+10,4.830400e+10,6.093200e+10,4.946000e+09,2.122300e+10,1.354050e+11,4.211700e+10,0.0,0.0,...,3.020830e+11,6.484900e+10,-3.068000e+09,-1.110900e+10,0.0,5.067200e+10,3.527550e+11,1.454630e+11,1.200690e+11,9.642300e+10
2022-06-25,2.750200e+10,2.072900e+10,4.823100e+10,4.224200e+10,5.433000e+09,1.638600e+10,1.122920e+11,4.033500e+10,0.0,0.0,...,2.782020e+11,6.211500e+10,5.289000e+09,-9.297000e+09,0.0,5.810700e+10,3.363090e+11,1.518060e+11,1.196910e+11,9.218900e+10
2022-03-26,2.809800e+10,2.341300e+10,5.151100e+10,4.540000e+10,5.460000e+09,1.580900e+10,1.181800e+11,3.930400e+10,0.0,0.0,...,2.832630e+11,6.118100e+10,1.271200e+10,-6.494000e+09,0.0,6.739900e+10,3.506620e+11,1.646320e+11,1.199810e+11,9.188300e+10
2021-12-25,3.711900e+10,2.679400e+10,6.391300e+10,6.525300e+10,5.876000e+09,1.811200e+10,1.531540e+11,3.924500e+10,0.0,0.0,...,3.092590e+11,5.842400e+10,1.443500e+10,-9.270000e+08,0.0,7.193200e+10,3.811910e+11,1.654770e+11,1.227980e+11,8.567900e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989-09-30,4.383000e+08,0.000000e+00,4.383000e+08,7.928000e+08,4.754000e+08,5.879000e+08,2.294400e+09,3.342000e+08,0.0,0.0,...,1.258200e+09,0.000000e+00,1.175900e+09,-3.091000e+08,618900000.0,1.485700e+09,2.743900e+09,0.000000e+00,5.680000e+07,-3.815000e+08
1988-09-30,5.457000e+08,0.000000e+00,5.457000e+08,6.388000e+08,4.615000e+08,1.370000e+08,1.783000e+09,2.074000e+08,0.0,0.0,...,1.078700e+09,0.000000e+00,7.765000e+08,-2.130000e+08,439900000.0,1.003400e+09,2.082100e+09,0.000000e+00,1.279000e+08,-4.178000e+08
1987-09-30,5.651000e+08,0.000000e+00,5.651000e+08,4.056000e+08,2.258000e+08,1.109000e+08,1.307400e+09,1.304000e+08,0.0,0.0,...,6.414000e+08,0.000000e+00,5.731000e+08,-1.587000e+08,422100000.0,8.365000e+08,1.477900e+09,0.000000e+00,0.000000e+00,-5.651000e+08
1986-09-30,5.762000e+08,0.000000e+00,5.762000e+08,2.631000e+08,1.087000e+08,9.290000e+07,1.040900e+09,1.073000e+08,0.0,0.0,...,4.660000e+08,0.000000e+00,4.743000e+08,-1.147000e+08,334500000.0,6.941000e+08,1.160100e+09,0.000000e+00,0.000000e+00,-5.762000e+08
